# Exercise 2: Data Processing and Analysis

In [14]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
try:
    from gofer.ok import check
except:
    %pip install git+https://github.com/grading/gradememaybe.git
    from gofer.ok import check

Today we're working with simulated smartwatch health data. You'll notice when you read in the csv file that we have 7 columns: User ID, Heart Rate (BPM), Blood Oxygen Level (%), Step Count, Sleep Duration (hours), Activity Level, and Stress Level. We're going to go through and "clean up" the data so that we can calculate some basic statistics such as mean, median, minimum, and maximum for each variable. Run the cell below to read in the table saving it in the variable `smartwatch`. 

In [15]:
smartwatch = pd.read_csv("unclean_smartwatch_health_data.csv")
print(smartwatch)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
0      4174.0         58.939776               98.809650   5450.390578   
1         NaN               NaN               98.532195    727.601610   
2      1860.0        247.803052               97.052954   2826.521994   
3      2294.0         40.000000               96.894213  13797.338044   
4      2130.0         61.950165               98.583797  15679.067648   
...       ...               ...                     ...           ...   
9995   1524.0         78.819386               98.931927   2948.491953   
9996   4879.0         48.632659               95.773035   4725.623070   
9997   2624.0         73.834442               97.945874   2571.492060   
9998   4907.0               NaN               98.401058   3364.788855   
9999   4113.0         70.063864               98.475606    544.696104   

     Sleep Duration (hours) Activity Level Stress Level  
0         7.167235622316564  Highly Active            1  
1      

Let's start together with the Heart Rate column. Just looking at the preview table above we can see two things right away: (1) we have `NaN` values meaning data was not collected for those individuals, and (2) some of the heart rate values are abnormally high i.e. `247.803052`. Let's see what the entire range of values look like. 

In [16]:
print(smartwatch['Heart Rate (BPM)'].min(), smartwatch['Heart Rate (BPM)'].max())

40.0 296.5939695131042


**Question 1**: Set the variables below equal to the minimum heart rate and the maximum heart rate in the dataset. This is just practice for the method of checking answers as you go. Afterwards run the cell below to check your answer. 

In [17]:
minimumHeartRate = smartwatch['Heart Rate (BPM)'].min()
maximumHeartRate = smartwatch['Heart Rate (BPM)'].max()

In [18]:
check('tests/q1.py')

KeyError: 'image/svg+xml'

Notice that the maximum value of `296` beats per minimute is WAY above normal range of heart rates. In fact, according to [heart.org](https://www.heart.org/en/healthy-living/fitness/fitness-basics/target-heart-rates) the highest estimated heart rate based on age ranges from 150 to 200 for adults. This will vary between individuals, but this is a good starting point for us to think about outliers in the heart rate values in this dataset. Let's see how many rows have missing data or heart rates above 200. 

In [19]:
## select rows where Heart Rate is NaN or rows where the heart rate is above 200. 
# Emphasis on the usage of 'or' here, we want rows where either 
# scenario 1 (NaN) OR scenario 2 (>200) is true. 

outlierHeartRaterows = smartwatch[smartwatch['Heart Rate (BPM)'] > 200]
nullHeartRaterows = smartwatch[smartwatch['Heart Rate (BPM)'].isnull()]

print(len(outlierHeartRaterows) + len(nullHeartRaterows))

450


In [20]:
## Now let's get a table of all the other rows. 
heartrateRows = smartwatch[(~smartwatch['Heart Rate (BPM)'].isnull()) & (smartwatch['Heart Rate (BPM)'] <= 200)]

print(heartrateRows)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
0      4174.0         58.939776               98.809650   5450.390578   
3      2294.0         40.000000               96.894213  13797.338044   
4      2130.0         61.950165               98.583797  15679.067648   
5      2095.0         96.285938               94.202910  10205.992256   
6      4772.0         47.272257               95.389760   3208.781177   
...       ...               ...                     ...           ...   
9994   1942.0         77.912299               98.640583  10061.145291   
9995   1524.0         78.819386               98.931927   2948.491953   
9996   4879.0         48.632659               95.773035   4725.623070   
9997   2624.0         73.834442               97.945874   2571.492060   
9999   4113.0         70.063864               98.475606    544.696104   

     Sleep Duration (hours) Activity Level Stress Level  
0         7.167235622316564  Highly Active            1  
3      

**Question 2:** 

Notice here that the length of the two tables (`450` and `9550`) add up to the total number rows (`n=10000`). This is a good sanity check as we manipulate the table. Now we have to decide how we deal with these missing values and outliers. One method to do this would be remove all the rows with null values or outlier values. Another method is use imputation - this can be done in several ways but below we're going to substitute the average heart rate for the missing and mismeasured values. Do we think this will change the mean?

In [21]:
##imputation
HeartRateMean = heartrateRows['Heart Rate (BPM)'].mean()
print(HeartRateMean)


75.13268404820141


In [22]:
outlierHeartRaterows['Heart Rate (BPM)'] = HeartRateMean
nullHeartRaterows['Heart Rate (BPM)'] = HeartRateMean


print(outlierHeartRaterows)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
2      1860.0         75.132684               97.052954   2826.521994   
337    2369.0         75.132684               95.441773   2998.761919   
393    2443.0         75.132684               95.497181   1227.804688   
403    3200.0         75.132684               96.011492   9402.746140   
595    2129.0         75.132684               97.161853   2555.402184   
649    2008.0         75.132684               98.356789   2739.171166   
818    3156.0         75.132684                     NaN   7281.778831   
1195   3261.0         75.132684               99.652006   2867.872064   
1391   4621.0         75.132684               96.688083  20577.677290   
1602   4737.0         75.132684               95.095839  16072.283561   
2023      NaN         75.132684               99.032130  17620.765455   
2211   2711.0         75.132684               97.852781   1786.998129   
2212   4020.0         75.132684               95.28

/tmp/ipykernel_2313/936369900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlierHeartRaterows['Heart Rate (BPM)'] = HeartRateMean
/tmp/ipykernel_2313/936369900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nullHeartRaterows['Heart Rate (BPM)'] = HeartRateMean


In [23]:
smartwatch_hr = pd.concat([outlierHeartRaterows, nullHeartRaterows, heartrateRows])

fullTableHRMean = smartwatch_hr['Heart Rate (BPM)'].mean()
print(fullTableHRMean)

75.13268404820141


In [24]:
check('tests/q2.py')

KeyError: 'image/svg+xml'

Notice how the mean doesn't change when you use the imputed mean as the substituted values. However, as a note this will change the distribution of values and has the potential to obscure the causes of missing values or outliers. 

In [25]:
sns.histplot(smartwatch['Heart Rate (BPM)'], label ='With Missing Values and Outliers')
sns.histplot(smartwatch_hr['Heart Rate (BPM)'], label = 'With Mean as Imputed Value')
plt.legend()

NameError: name '_fetch_figure_metadata' is not defined

**Question 3**: How does the imputation method affect the median values?  Remember the table `smartwatch` remains unchanged and can be used to find the original median value. 

ANSWER: the imputation slightly decreased the median from 75.22060125775644 to the imputed value (75.132684)


In [26]:
## coding cell to check the difference in median values 

print(smartwatch['Heart Rate (BPM)'].median(), smartwatch_hr['Heart Rate (BPM)']. median())

75.22060125775644 75.13268404820141


Now let's repeat this process for the other columns as well. 

**Question 4**: Find the minimum, maximum, and mean Blood Oxygen Level. 

In [27]:
minBloodO2 = smartwatch['Blood Oxygen Level (%)'].min()

maxBloodO2 = smartwatch['Blood Oxygen Level (%)'].max()

meanBloodO2 = smartwatch['Blood Oxygen Level (%)'].mean()

print(minBloodO2, maxBloodO2, meanBloodO2)

90.79120814564097 100.0 97.84158102099076


In [28]:
check('tests/q4.py')

KeyError: 'image/svg+xml'

We can do some visualizations using a box plot to better decide if there's any outliers we'd like to remove. Maybe you decide that everything below 92.5 is an outlier should be removed or maybe you decide to keep all of the values or maybe you decide to remove all values under 94% since that falls outside of a normal, healthy range according to doctors (I would not suggest this last one as it would obscure quite a bit of data! But some analysts might consider it!). In question 5, you can make that decision and justify your answer. 

In [29]:
sns.boxplot(smartwatch['Blood Oxygen Level (%)'])

NameError: name '_fetch_figure_metadata' is not defined

**Question 5**: Decide how you want to deal with missing and outlier values in the Blood Oxygen Level column. Set your final table equal to the variable `smartwatch_o2`. Use the space below to explain your decisions. 

Notes: Refer to [this article](https://pmc.ncbi.nlm.nih.gov/articles/PMC5548942/) on methods of handling these type of instances in data. Keep in mind, you might find that either there are no missing values or no outliers. That's okay, just indicate that in the written space below and update the table i.e. `smartwatch_o2 = smartwatch_hr`

ANSWER: beacuse the data is being collected by a smart watch, it is likely the data is missing completely at random due to malfuncitoning software or hardware, so it is justifyable to use mean imputation, however multiple imputation would likely be a better option. Because we have not discussed multiple imputation in class i will stick to mean imputation. Additionally, given that there don't appear to be any outliers, I will only be imputing to null values

....

In [30]:
## use this space or additional cells to deal with the missing/outlier values. 

nullBO2 = smartwatch_hr[smartwatch_hr['Blood Oxygen Level (%)'].isnull()]
BO2Rows = smartwatch_hr[(~smartwatch_hr['Blood Oxygen Level (%)'].isnull())]
BO2Mean = BO2Rows['Blood Oxygen Level (%)'].mean()
print(BO2Mean)
nullBO2['Blood Oxygen Level (%)'] = BO2Mean
print(nullBO2)

smartwatch_o2 = pd.concat([nullBO2, BO2Rows])

fullTableBO2Mean = smartwatch_o2['Blood Oxygen Level (%)'].mean()
print(fullTableBO2Mean)

print(smartwatch_o2)



97.84158102099076
      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
818    3156.0         75.132684               97.841581   7281.778831   
8369   2283.0         75.132684               97.841581  21495.639335   
1615   1417.0         75.132684               97.841581   1585.892722   
1712   4079.0         75.132684               97.841581  19003.921284   
4690   2829.0         75.132684               97.841581   6610.794796   
...       ...               ...                     ...           ...   
9834   4840.0         64.803112               97.841581   1647.308449   
9861   3942.0         57.617790               97.841581  14269.737149   
9885   4028.0         75.281591               97.841581   3852.160740   
9905   3603.0         87.693723               97.841581   2364.028205   
9979   3631.0         58.797407               97.841581    359.210435   

     Sleep Duration (hours) Activity Level Stress Level  
818       4.300171191258864      Sedentary     

/tmp/ipykernel_2313/4081464534.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nullBO2['Blood Oxygen Level (%)'] = BO2Mean


In [31]:
minBloodO2_imp = smartwatch_o2['Blood Oxygen Level (%)'].min()

maxBloodO2_imp = smartwatch_o2['Blood Oxygen Level (%)'].max()

meanBloodO2_imp = smartwatch_o2['Blood Oxygen Level (%)'].mean()

print(minBloodO2_imp, maxBloodO2_imp, meanBloodO2_imp)

90.79120814564097 100.0 97.84158102099076


Moving onto the Step Count column. 

**Question 6**: Find the minimum, maximum, and mean step counts. 

In [32]:
minSteps = smartwatch['Step Count'].min()

maxSteps = smartwatch['Step Count'].max()

meanSteps = smartwatch['Step Count'].mean()

print(minSteps, maxSteps, meanSteps)

0.9101380609604088 62486.690753464914 6985.685884992229


In [33]:
check('tests/q6.py')

KeyError: 'image/svg+xml'

**Question 7**: Decide how you want to deal with missing and outlier values in the Steps column. Set your final table equal to the variable `smartwatch_steps`. Use the space below to explain your decisions. 

Notes: Refer to [this article](https://pmc.ncbi.nlm.nih.gov/articles/PMC5548942/) on methods of handling these type of instances in data. Keep in mind, you might find that either there are no missing values or no outliers. That's okay, just indicate that in the written space below and update the table i.e. `smartwatch_steps = smartwatch_o2`

ANSWER: The first roadblock i hit was weather to count the extreme values as outliers or not. It is reasonable that someone who may be bedridden or otherwise disabled could record 0.9 average steps in a week, alternatively it is possible that (for example) someone training to run a marathon could average ~62,000 steps. Given this dilema, I have decided not to impute outliers because I do not know if they are truly measurement error or simply extreme circumstance. 

....

In [34]:
## use this space or additional cells to address the missing or outlier data. 

nullStepRows = smartwatch_o2[smartwatch_o2['Step Count'].isnull()]
#print(len(nullStepRows))
StepRows = smartwatch_o2[(~smartwatch_o2['Step Count'].isnull())]
#print(len(StepRows))
StepMean = StepRows['Step Count'].mean()
#print(StepMean)

nullStepRows['Step Count'] = StepMean
#print(nullStepRows)

smartwatch_step = pd.concat([nullStepRows, StepRows])

fullTableStepMean = smartwatch_step['Step Count'].mean()

print(smartwatch_step)

#smartwatch_steps = ...

#smartwatch_steps

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
1509   4719.0        107.126196               97.841581   6985.685885   
5870   1499.0         75.132684               96.833301   6985.685885   
204    2923.0         75.132684               98.710637   6985.685885   
841    4301.0         75.132684               98.267833   6985.685885   
2798   4923.0         75.132684              100.000000   6985.685885   
...       ...               ...                     ...           ...   
9994   1942.0         77.912299               98.640583  10061.145291   
9995   1524.0         78.819386               98.931927   2948.491953   
9996   4879.0         48.632659               95.773035   4725.623070   
9997   2624.0         73.834442               97.945874   2571.492060   
9999   4113.0         70.063864               98.475606    544.696104   

     Sleep Duration (hours) Activity Level Stress Level  
1509       7.21392401415147      Sedentary            2  
5870   

/tmp/ipykernel_2313/1556064856.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nullStepRows['Step Count'] = StepMean


Next onto the Sleep Duration column. 

**Question 8**: Try finding the minimum number of hours slept among participants. 

In [35]:
minSleep = smartwatch['Sleep Duration (hours)'].min()

print(minSleep)

TypeError: '<=' not supported between instances of 'str' and 'float'

In the error message, you should see the phrase: 

`TypeError: '<=' not supported between instances of 'str' and 'float'`

This means that the column is a combination of multiple data types. If you recall, we discussed computer readable data that columns HAVE to contain just a single data type. Having a combination of strings, characters, and numbers in a column will only cause more issues downstream. Let's try to figure out all the instances of non-numerical values in the column. 

First let's try to identify all the non-numerical values to make sure removing them doesn't obscure any important information. 

In [36]:
for value in smartwatch['Sleep Duration (hours)'].unique():  ## Loop through each unique item in the column
    try:
        numericVal = float(value)  ## first try to see if it can be converted to a number
    except:
        print(value)  ## if it can't be converted print it to screen 

ERROR


So, we find that the only non-numerical value is the string `ERROR`. We can fix this in two ways. First let's try fixing it by splitting the tables like we've done previously. 

In [37]:
errorTable = smartwatch_step[smartwatch_step['Sleep Duration (hours)'] == 'ERROR']
print(errorTable)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
8642   3476.0         75.132684               99.462778   6985.685885   
1456   2997.0         54.116580               95.631824   6985.685885   
4349   2677.0         73.357223               95.335209   6985.685885   
6992   2105.0         97.477048               94.138582   6985.685885   
711    2066.0         64.843081               97.841581  12722.224091   
...       ...               ...                     ...           ...   
9901   2269.0         63.369107               96.229029   6656.786482   
9912   2074.0         75.933666               97.386073   1813.072248   
9927   3007.0         87.579045               98.768633   3727.161872   
9943   2961.0         91.834863               96.145378  38831.465057   
9951   3104.0         70.335328               98.560133   1997.442874   

     Sleep Duration (hours) Activity Level Stress Level  
8642                  ERROR         Active            3  
1456   

Observe that we've created a table with the rows with `ERROR` in the sleep duration column. We can now replace with the `ERROR` value with our handy NaN value for doing numerical statistics. 

In [38]:
errorTable['Sleep Duration (hours)'] = np.nan

print(errorTable)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
8642   3476.0         75.132684               99.462778   6985.685885   
1456   2997.0         54.116580               95.631824   6985.685885   
4349   2677.0         73.357223               95.335209   6985.685885   
6992   2105.0         97.477048               94.138582   6985.685885   
711    2066.0         64.843081               97.841581  12722.224091   
...       ...               ...                     ...           ...   
9901   2269.0         63.369107               96.229029   6656.786482   
9912   2074.0         75.933666               97.386073   1813.072248   
9927   3007.0         87.579045               98.768633   3727.161872   
9943   2961.0         91.834863               96.145378  38831.465057   
9951   3104.0         70.335328               98.560133   1997.442874   

      Sleep Duration (hours) Activity Level Stress Level  
8642                     NaN         Active            3  
1456 

/tmp/ipykernel_2313/2212040875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errorTable['Sleep Duration (hours)'] = np.nan


Now we could go through and put the tables back together, but another method to do this is to use a built-in function to pandas called `pd.to_numeric()`. Let's try that. 

In [39]:
## create a copy of the table 
smartwatch_sleep = smartwatch_step

smartwatch_sleep['Sleep Duration (hours)'] = pd.to_numeric(smartwatch_sleep['Sleep Duration (hours)'], errors='coerce')

print(smartwatch_sleep)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
1509   4719.0        107.126196               97.841581   6985.685885   
5870   1499.0         75.132684               96.833301   6985.685885   
204    2923.0         75.132684               98.710637   6985.685885   
841    4301.0         75.132684               98.267833   6985.685885   
2798   4923.0         75.132684              100.000000   6985.685885   
...       ...               ...                     ...           ...   
9994   1942.0         77.912299               98.640583  10061.145291   
9995   1524.0         78.819386               98.931927   2948.491953   
9996   4879.0         48.632659               95.773035   4725.623070   
9997   2624.0         73.834442               97.945874   2571.492060   
9999   4113.0         70.063864               98.475606    544.696104   

      Sleep Duration (hours) Activity Level Stress Level  
1509                7.213924      Sedentary            2  
5870 

Now we can finish **Question 8** and find the minimum, maximum, and mean of the sleep duration column. 

In [40]:
minSleep = smartwatch_sleep['Sleep Duration (hours)'].min()

maxSleep = smartwatch_sleep['Sleep Duration (hours)'].max()

meanSleep = smartwatch_sleep['Sleep Duration (hours)'].mean()

print(minSleep, maxSleep, meanSleep)

-0.1944527906201543 12.140232872862926 6.505462918406444


In [ ]:
check('tests/q8.py')

KeyError: 'image/svg+xml'

**Question 9**: Decide how you want to deal with missing and outlier values in the sleep column. Set your final table equal to the variable `smartwatch_updated_sleep`. Use the space below to explain your decisions. 

*Remember to start with the `smartwatch_sleep` table that we just created.*

Notes: Refer to [this article](https://pmc.ncbi.nlm.nih.gov/articles/PMC5548942/) on methods of handling these type of instances in data. Keep in mind, you might find that either there are no missing values or no outliers. That's okay, just indicate that in the written space below and update the table i.e. `smartwatch_updated_sleep = smartwatch_sleep`

ANSWER: the minimum number of hours a person can sleep for per night and survive is roughly 5 (https://time.com/6994478/how-much-sleep-do-you-need/) so i will set that as my outlier minimum. I am not going to eliminate any higher end value as they are more reasonable than the lower ones. I use mean imputation fot the rest of the missing variables as it is likely they are a result of user or machine error (likely a user taking their watch off)

....

In [44]:
## use this space or additional cells to address the missing or outlier data. 
nullSleepRows = smartwatch_sleep[smartwatch_sleep['Sleep Duration (hours)'].isnull()]
outlierSleepRows = smartwatch_sleep[smartwatch_sleep['Sleep Duration (hours)'] < 5]
print(len(nullSleepRows), len(outlierSleepRows))
SleepRows = smartwatch_sleep[(~smartwatch_sleep['Sleep Duration (hours)'].isnull()) & (smartwatch_sleep['Sleep Duration (hours)'] >= 5)]
SleepMean = SleepRows['Sleep Duration (hours)'].mean()
print(SleepMean)
nullSleepRows['Sleep Duration (hours)'] = SleepMean

smartwatch_updated_sleep = pd.concat([nullSleepRows, outlierSleepRows, SleepRows])
print(smartwatch_updated_sleep)


397 1527
6.93841861521835
      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
8642   3476.0         75.132684               99.462778   6985.685885   
1456   2997.0         54.116580               95.631824   6985.685885   
4349   2677.0         73.357223               95.335209   6985.685885   
5177   2997.0         70.484333               97.615592   6985.685885   
6992   2105.0         97.477048               94.138582   6985.685885   
...       ...               ...                     ...           ...   
9994   1942.0         77.912299               98.640583  10061.145291   
9995   1524.0         78.819386               98.931927   2948.491953   
9996   4879.0         48.632659               95.773035   4725.623070   
9997   2624.0         73.834442               97.945874   2571.492060   
9999   4113.0         70.063864               98.475606    544.696104   

      Sleep Duration (hours) Activity Level Stress Level  
8642                6.938419         A

/tmp/ipykernel_2313/3693829026.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nullSleepRows['Sleep Duration (hours)'] = SleepMean


We're going to skip the `Activity Level` column for a minute and look at the `Stress Level` column. If we try getting the minimum, we'll find the same error as in the Sleep column where we have mixed data types (strings and numerical values). Let's use the same type of loop to make sure we don't obscure any data by forcing the strings to NaN values. 

In [42]:
for item in smartwatch_updated_sleep['Stress Level'].unique():
    try:
        int(item)
    except:
        print(item)

Very High
nan


**Question 10**: 

What might you decide to do to deal with the `Very High` value? 

Go ahead and do so below and give a brief case for doing so here. Assign your table to the variable `smartwatch_stress`. 

ANSWER: Given that the stress level range appears to be 1-10, i would assign "Very high" to a 9. 

TypeError: '<=' not supported between instances of 'str' and 'float'

In [48]:
## cell to deal with 'Very High' value
smartwatch_updated_sleep['Stress Level'] = smartwatch_updated_sleep['Stress Level'].replace({ "Very High": 9, "nan": np.nan})
##cited, chat gpt "i got this error message [pasted message] please debug"
smartwatch_updated_sleep['Stress Level'] = pd.to_numeric(smartwatch_updated_sleep['Stress Level'], errors='coerce')

nullStressRows = smartwatch_updated_sleep[smartwatch_updated_sleep['Stress Level'].isnull()]
print(len(nullStressRows))
StressRows = smartwatch_updated_sleep[(~smartwatch_updated_sleep['Stress Level'].isnull())]
StressMean = StressRows['Stress Level'].mean()
print(StressMean)
nullStressRows['Stress Level'] = StressMean

smartwatch_stress = pd.concat([nullStressRows, StressRows])
print(smartwatch_stress)

200
5.481734693877551
      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
3919      NaN         46.812987               95.234249   7893.183565   
5596   3871.0         71.320880               99.228302   3498.719211   
6127   4821.0         89.024555               96.138871   4261.951634   
6165   4601.0         79.244143               96.013915   2311.324740   
7383   1871.0         61.256883               97.491386   5085.035828   
...       ...               ...                     ...           ...   
9994   1942.0         77.912299               98.640583  10061.145291   
9995   1524.0         78.819386               98.931927   2948.491953   
9996   4879.0         48.632659               95.773035   4725.623070   
9997   2624.0         73.834442               97.945874   2571.492060   
9999   4113.0         70.063864               98.475606    544.696104   

      Sleep Duration (hours) Activity Level  Stress Level  
3919                6.938419  Highly_Acti

/tmp/ipykernel_2313/3110481679.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nullStressRows['Stress Level'] = StressMean


Finally, let's go back to the `Activity Level` column and investigate what types of values we find there. 

In [49]:
print(smartwatch['Activity Level'].unique())

['Highly Active' 'Highly_Active' 'Actve' 'Seddentary' 'Sedentary' 'Active'
 nan]


**Question 12**: 

What do you notice? There are several values that could and should be combined because they represent the same information. Let's go ahead and do that. While combining these columns, let's also create a new column `NumActivity` where we give a numerical value to represent the activity level. Assign your final table to the variable `final_table`. 

`Highly Active` = `1`

`Active` = `2`

`Sedentary` = `3`



In [50]:
## Highly Active 
highlyActive = smartwatch_stress[(smartwatch_stress['Activity Level'] == 'Highly Active') | (smartwatch_stress['Activity Level'] == 'Highly_Active')]  ## Pull out rows for two unique values matching highly active
highlyActive['Activity Level'] = 'Highly Active'  ## Reset the text in the column to either 'Highly Active'
highlyActive['NumActivity'] = 1 ## set the number for the numerical value 


## Active
active = smartwatch_stress[(smartwatch_stress['Activity Level'] == 'Active') | (smartwatch_stress['Activity Level'] == 'Actve')]
active['Activity Level'] = 'Active'
active['NumActivity'] = 2 

## Sedentary 
sedentary = smartwatch_stress[(smartwatch_stress['Activity Level'] == 'Sedentary') | (smartwatch_stress['Activity Level'] == 'Seddentary')]  
sedentary['Activity Level'] = 'Sedentary'  
sedentary['NumActivity'] = 3 

final_table = pd.concat([highlyActive, active, sedentary, smartwatch_stress[smartwatch_stress['Activity Level'].isnull()]])
print(final_table)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
3919      NaN         46.812987               95.234249   7893.183565   
7383   1871.0         61.256883               97.491386   5085.035828   
8774   4870.0         52.634964               97.971538  15906.886529   
1660   1989.0         68.138095              100.000000    332.500417   
1708   2823.0         66.209200              100.000000   4998.155811   
...       ...               ...                     ...           ...   
9814   1630.0         58.877709              100.000000   7357.627891   
9842   2108.0         71.152455               98.585987   1480.777008   
9845   4203.0         91.417098               95.383875   1238.788704   
9898   1409.0         79.941705               98.716816    619.353083   
9991   4289.0         91.381582               95.741838   3930.126338   

      Sleep Duration (hours) Activity Level  Stress Level  NumActivity  
3919                6.938419  Highly Active      5

/tmp/ipykernel_2313/2281820914.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highlyActive['Activity Level'] = 'Highly Active'  ## Reset the text in the column to either 'Highly Active'
/tmp/ipykernel_2313/2281820914.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highlyActive['NumActivity'] = 1 ## set the number for the numerical value
/tmp/ipykernel_2313/2281820914.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

Let's check to make sure that we no longer have any missing values in each column (besides the Activity Level/NumActivity, Stress Level, and User ID columns). You likely either removed those rows or imputed a value to substitute the missing values. 

In [ ]:
print(final_table.isnull().sum())

User ID                   201
Heart Rate (BPM)            0
Blood Oxygen Level (%)      0
Step Count                  0
Sleep Duration (hours)      0
Activity Level            200
Stress Level                0
NumActivity               200
dtype: int64


Then let's use the info function to make sure each column has the data type we're expecting. 

In [ ]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 3919 to 9991
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 9799 non-null   float64
 1   Heart Rate (BPM)        9600 non-null   float64
 2   Blood Oxygen Level (%)  9700 non-null   float64
 3   Step Count              10000 non-null  float64
 4   Sleep Duration (hours)  10000 non-null  float64
 5   Activity Level          9800 non-null   object 
 6   Stress Level            10000 non-null  float64
 7   NumActivity             9800 non-null   float64
dtypes: float64(7), object(1)
memory usage: 703.1+ KB


**Question 13**: 

Let's visualize two of the variables. Let's pick `Heart Rate (BPM)` and then you can select any other numerical variable. We're going to create a scatter plot using matplotlib.pyplot. Example code is: 

`plt.scatter(x, y)` where x and y are your columns of data such as df['label']

Also try including a size parameter to make your points smaller to better see patterns. We'll talk more about creating figures in python in a few weeks, but for now let's just look at the broad patterns. 

Example of including size parameter: 

`plt.scatter(x, y, s=1)` Try changing the `s` parameter to 10, 1, 0.5, and 0.1. 

What do you notice?

ANSWER: Scatterplot command is still not working with the packages 

...

In [ ]:
plt.scatter(final_table['Heart Rate (BPM)', final_table['Stress Level']], s=1)

InvalidIndexError: ('Heart Rate (BPM)', 3919    5.481735
7383    5.481735
8774    5.481735
1660    5.481735
1708    5.481735
          ...   
9814    5.000000
9842    9.000000
9845    7.000000
9898    8.000000
9991    8.000000
Name: Stress Level, Length: 10000, dtype: float64)

**Question 14**: 


Read in the CSV table where we kept all the NaN values instead of removing or imputing them. Repeat the exact same visualization as above with this data. 

Compare the two figures. What do you notice?

ANSWER: scatterplot command still not working with packages

...

In [51]:
nanTable = pd.read_csv('smartwatch_nan_vals.csv')
print(nanTable)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
0      4670.0         70.659253               99.072904   6042.576181   
1      1726.0         91.127561              100.000000   4213.519341   
2      4627.0         74.776893               99.630704  12557.592821   
3      1556.0         91.216912               98.777090  50224.691117   
4      3320.0         66.331358               99.903851    819.769598   
...       ...               ...                     ...           ...   
9995   2597.0         80.728128               97.254023           NaN   
9996   2096.0         57.087738               98.961619           NaN   
9997   2577.0         65.201322               99.484801   2240.504798   
9998   3501.0         76.063875               96.130100  12510.840514   
9999   3895.0         78.398919              100.000000   2522.668511   

      Sleep Duration (hours) Activity Level  Stress Level  NumActivity  
0                   6.453973  Highly Active       

In [52]:
plt.scatter(nanTable['Heart Rate (BPM)', final_table['Stress Level']], s=1)

InvalidIndexError: ('Heart Rate (BPM)', 3919    5.481735
7383    5.481735
8774    5.481735
1660    5.481735
1708    5.481735
          ...   
9814    5.000000
9842    9.000000
9845    7.000000
9898    8.000000
9991    8.000000
Name: Stress Level, Length: 10000, dtype: float64)

**Question 15**: 


Does the amount of steps signficantly differ between the different activity level groups? 

Hint: Try using a boxplot (`sns.boxplot`) to first visualize the problem. Then you can use scipy.stats to run an ANOVA. 

In [53]:
## Visualization 

sns.boxplot(nanTable['Activity Level'])


NameError: name '_fetch_figure_metadata' is not defined

In [54]:
from scipy.stats import f_oneway

## create a table for each activity level group
sed = final_table[final_table['Activity Level'] == 'Sedentary'] ## sedentary rows
act = final_table[final_table['Activity Level'] == 'Active'] ## active rows
hact = final_table[final_table['Activity Level'] == 'Highly Active'] ## highly active rows 

stat, pval = f_oneway(sed['Step Count'], act['Step Count'], hact['Step Count'])


print(f"ANOVA test statistic: {stat:.3f}, p-value: {pval:.3f}")


ANOVA test statistic: 0.155, p-value: 0.857


Is there significant difference between the groups' step counts?  

ANSWER: no, with a test statistic of 0.155 and a p-value there we would not be able to reasonably reject the null that there is no relation between steps and activity level. 

**Question 16**

If you were to present your findings from this dataset to a broader audience (such as policymakers, healthcare providers, or the general public), how would you communicate key insights responsibly? What considerations would you take into account to avoid misrepresenting the data or reinforcing biases?

ANSWER: i would most importantly take into consideration that there is little to no information about how this data was collected, who the test subjects were, or how well the hardware functions. I would also be sure to inform them that there is a lot of missing data and that mean imputation is not necessarily the best way to deal with that missing data. Likely this data set is not a great representation of reality and really should not be used to inform policy or medical practices.

....